In [21]:
import pandas as pd
import numpy as np
import optuna

In [22]:
xgb_df = pd.read_csv("/kaggle/input/xgboost/XGBsubmission.csv")
lgb_df = pd.read_csv("/kaggle/input/lgboostcsv/lgsubmission (1).csv")
cat_df = pd.read_csv("/kaggle/input/cat-bo/submission_catb.csv")
lr_df  = pd.read_csv("/kaggle/input/linearref/lrsubmission.csv")
gb_df  = pd.read_csv("/kaggle/input/grad-boost/submission(001).csv")

xgb_df = xgb_df.rename(columns={'Premium Amount': 'xgb_pred'})
lgb_df = lgb_df.rename(columns={'Premium Amount': 'lgb_pred'})
cat_df = cat_df.rename(columns={'Premium Amount': 'cat_pred'})
lr_df  = lr_df.rename(columns={'Premium Amount': 'lr_pred'})
gb_df  = gb_df.rename(columns={'Premium Amount': 'gb_pred'})


In [23]:
gb_df.head()

,id,gb_pred
0,1200000,415.584163
1,1200001,604.458277
2,1200002,588.745439
3,1200003,588.745439
4,1200004,571.045783


In [24]:
lr_df.head()

,id,lr_pred
0,1200000,907.222200
1,1200001,582.835247
2,1200002,674.658230
3,1200003,643.867759
4,1200004,696.832252


In [25]:
lgb_df.head()

,id,lgb_pred
0,1200000,718.528380
1,1200001,863.284798
2,1200002,799.298493
3,1200003,789.131510
4,1200004,778.381459


In [26]:
xgb_df.head()

,id,xgb_pred
0,1200000,714.507959
1,1200001,829.446521
2,1200002,785.417798
3,1200003,804.212048
4,1200004,773.894006


In [27]:
cat_df.head()

,id,cat_pred
0,1200000,778.918317
1,1200001,784.389178
2,1200002,785.537065
3,1200003,782.219267
4,1200004,764.650581


In [28]:
combined_df = pd.concat([xgb_df, lgb_df, cat_df, lr_df, gb_df], axis = 1)
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
combined_df.head()

,id,xgb_pred,lgb_pred,cat_pred,lr_pred,gb_pred
0,1200000,714.507959,718.528380,778.918317,907.222200,415.584163
1,1200001,829.446521,863.284798,784.389178,582.835247,604.458277
2,1200002,785.417798,799.298493,785.537065,674.658230,588.745439
3,1200003,804.212048,789.131510,782.219267,643.867759,588.745439
4,1200004,773.894006,778.381459,764.650581,696.832252,571.045783


In [29]:
import numpy as np
import pandas as pd

# Assuming your DataFrame is called 'df'
def exponential_mean_log1p(predictions):
    # Apply log1p transformation
    log_preds = np.log1p(predictions)
    
    # Calculate mean of log-transformed predictions
    mean_log_preds = np.mean(log_preds, axis=1)
    
    # Apply exponential (inverse of log1p)
    final_predictions = np.expm1(mean_log_preds)
    
    return final_predictions


# Select prediction columns
pred_columns = ['xgb_pred', 'lgb_pred', 'cat_pred', 'lr_pred', 'gb_pred']

# Calculate final predictions
combined_df['final_pred'] = exponential_mean_log1p(combined_df[pred_columns])

# Display the results
combined_df.head()

,id,xgb_pred,lgb_pred,cat_pred,lr_pred,gb_pred,final_pred
0,1200000,714.507959,718.528380,778.918317,907.222200,415.584163,684.995516
1,1200001,829.446521,863.284798,784.389178,582.835247,604.458277,723.245153
2,1200002,785.417798,799.298493,785.537065,674.658230,588.745439,721.775192
3,1200003,804.212048,789.131510,782.219267,643.867759,588.745439,716.010727
4,1200004,773.894006,778.381459,764.650581,696.832252,571.045783,712.248657


In [30]:
import numpy as np
import pandas as pd

def average_predictions(df, pred_columns):
    return np.mean(df[pred_columns], axis=1)

pred_columns = ['xgb_pred', 'lgb_pred', 'cat_pred', 'lr_pred', 'gb_pred']

# Calculate average predictions
combined_df['avg_predictions'] = average_predictions(combined_df, pred_columns)

# Display results
combined_df.head()

,id,xgb_pred,lgb_pred,cat_pred,lr_pred,gb_pred,final_pred,avg_predictions
0,1200000,714.507959,718.528380,778.918317,907.222200,415.584163,684.995516,706.952204
1,1200001,829.446521,863.284798,784.389178,582.835247,604.458277,723.245153,732.882804
2,1200002,785.417798,799.298493,785.537065,674.658230,588.745439,721.775192,726.731405
3,1200003,804.212048,789.131510,782.219267,643.867759,588.745439,716.010727,721.635205
4,1200004,773.894006,778.381459,764.650581,696.832252,571.045783,712.248657,716.960816


In [31]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
sub['Premium Amount'] = combined_df['final_pred']
sub.to_csv('submission.csv', index=False)
sub.head()

,id,Premium Amount
0,1200000,684.995516
1,1200001,723.245153
2,1200002,721.775192
3,1200003,716.010727
4,1200004,712.248657


In [32]:
xgb_oof_df = pd.read_csv("/kaggle/input/xgboost/oof_xgb.csv").drop(columns =['Actual','Fold'])
lgb_oof_df = pd.read_csv("/kaggle/input/lgboostcsv/oof_lgbm.csv").drop(columns =['Actual','Fold'])
cat_oof_df = pd.read_csv("/kaggle/input/cat-bo/oof_catb.csv").drop(columns =['Fold'])
lr_oof_df = pd.read_csv("/kaggle/input/linearref/oof_lr.csv").drop(columns =['Actual','Fold'])
gb_oof_df = pd.read_csv("/kaggle/input/grad-boost/oof_aaba.csv").drop(columns =['Actual','Fold'])

In [33]:
xgb_oof_df.head()

,ID,OOF_Pred_XGB
0,4,784.983582
1,15,863.431091
2,21,715.363892
3,30,792.706299
4,34,764.853027


In [35]:
print(gb_oof_df.columns)


Index(['ID', 'OOF_Pred'], dtype='object')


In [36]:
# Merge DataFrames on 'ID'
combined_oof_df = xgb_oof_df.merge(lgb_oof_df, on='ID')
combined_oof_df = combined_oof_df.merge(cat_oof_df, on='ID')
combined_oof_df = combined_oof_df.merge(lr_oof_df, on='ID')  # Also merge the LR predictions
combined_oof_df = combined_oof_df.merge(gb_oof_df, on="ID")


# Select the required columns
combined_oof_df = combined_oof_df[['ID', 'OOF_Pred_XGB', 'OOF_Pred_LGB', 'OOF_Pred_cat', 'OOF_Pred_LR', 'OOF_Pred', 'Actual']]

# Display the final combined DataFrame
combined_oof_df.head()


,ID,OOF_Pred_XGB,OOF_Pred_LGB,OOF_Pred_cat,OOF_Pred_LR,OOF_Pred,Actual
0,4,784.983582,790.468576,764.757529,667.492085,591.994617,2022.0
1,15,863.431091,844.729374,862.122012,779.015390,573.931354,849.0
2,21,715.363892,695.666440,778.202261,654.119606,573.931354,2670.0
3,30,792.706299,835.328327,803.697458,789.055631,602.764959,641.0
4,34,764.853027,791.740481,777.046047,649.752725,591.994617,2152.0


In [37]:
combined_oof_df.head()

,ID,OOF_Pred_XGB,OOF_Pred_LGB,OOF_Pred_cat,OOF_Pred_LR,OOF_Pred,Actual
0,4,784.983582,790.468576,764.757529,667.492085,591.994617,2022.0
1,15,863.431091,844.729374,862.122012,779.015390,573.931354,849.0
2,21,715.363892,695.666440,778.202261,654.119606,573.931354,2670.0
3,30,792.706299,835.328327,803.697458,789.055631,602.764959,641.0
4,34,764.853027,791.740481,777.046047,649.752725,591.994617,2152.0


In [42]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split

def rmsle(y_true, y_pred):
    """
    Root Mean Squared Logarithmic Error
    Handles cases with zero and negative values
    """
    # Add small constant to avoid log(0)
    y_true = np.abs(y_true) + 1e-6
    y_pred = np.abs(y_pred) + 1e-6
    
    # Calculate log error
    log_error = np.log1p(y_true) - np.log1p(y_pred)
    
    # Calculate RMSLE
    return np.sqrt(np.mean(log_error**2))

def objective(trial):
    # Prepare the data
    X = df[['OOF_Pred_XGB', 'OOF_Pred_LGB', 'OOF_Pred_cat', 'OOF_Pred_LR', 'OOF_Pred']]  # Include all model predictions
    y = df['Actual']
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Optimize weights
    w1 = trial.suggest_float('w1', 0, 1)
    w2 = trial.suggest_float('w2', 0, 1)
    w3 = trial.suggest_float('w3', 0, 1)
    w4 = trial.suggest_float('w4', 0, 1)
    w5 = trial.suggest_float('w5', 0, 1)  # Added weight for GB predictions
    
    # Normalize weights
    total_weight = w1 + w2 + w3 + w4 + w5  # Total weight now includes w5 for GB
    weights = [w1/total_weight, w2/total_weight, w3/total_weight, w4/total_weight, w5/total_weight]  # Include GB weight
    
    # Calculate weighted prediction
    pred = np.average(X_test, axis=1, weights=weights)
    
    # Calculate RMSLE
    rmsle_score = rmsle(y_test, pred)
    
    return rmsle_score

# Merge DataFrames on 'ID'
combined_oof_df = xgb_oof_df.merge(lgb_oof_df, on='ID')
combined_oof_df = combined_oof_df.merge(cat_oof_df, on='ID')
combined_oof_df = combined_oof_df.merge(lr_oof_df, on='ID')  # Also merge the LR predictions
combined_oof_df = combined_oof_df.merge(gb_oof_df, on='ID')  # Merge GB predictions

# Select the required columns
combined_oof_df = combined_oof_df[['ID', 'OOF_Pred_XGB', 'OOF_Pred_LGB', 'OOF_Pred_cat', 'OOF_Pred_LR', 'OOF_Pred', 'Actual']]

# Display the final combined DataFrame
print(combined_oof_df.head())

# Create DataFrame (as in your previous example)
df = combined_oof_df

# Create a study object and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)

# Print results
print("Best trial:")
trial = study.best_trial
print("  Best RMSLE: ", trial.value)
print("  Best Weights:")
print(f"    XGB: {trial.params['w1']}")
print(f"    LGB: {trial.params['w2']}")
print(f"    Cat: {trial.params['w3']}")
print(f"    LR: {trial.params['w4']}")
print(f"    GB: {trial.params['w5']}")  # Print GB weight

# Final weighted prediction using best weights
best_w1 = trial.params['w1']
best_w2 = trial.params['w2']
best_w3 = trial.params['w3']
best_w4 = trial.params['w4']
best_w5 = trial.params['w5']  # Get best weight for GB
total_weight = best_w1 + best_w2 + best_w3 + best_w4 + best_w5  # Total weight includes GB
weights = [best_w1/total_weight, best_w2/total_weight, best_w3/total_weight, best_w4/total_weight, best_w5/total_weight]

# Final prediction using best weights
final_pred = np.average(df[['OOF_Pred_XGB', 'OOF_Pred_LGB', 'OOF_Pred_cat', 'OOF_Pred_LR', 'OOF_Pred']], 
                        axis=1, 
                        weights=weights)
df['Weighted_Prediction'] = final_pred

# Display final results
print("\nFinal DataFrame with Weighted Prediction:")
print(df)


[I 2024-12-29 17:02:31,358] A new study created in memory with name: no-name-693a91aa-e457-44f6-b051-58fd4a59855f
[I 2024-12-29 17:02:31,519] Trial 0 finished with value: 1.0599131324428062 and parameters: {'w1': 0.6208116968927979, 'w2': 0.2781491438242467, 'w3': 0.9155113338226609, 'w4': 0.9981021475588872, 'w5': 0.9281419510318315}. Best is trial 0 with value: 1.0599131324428062.


   ID  OOF_Pred_XGB  OOF_Pred_LGB  OOF_Pred_cat  OOF_Pred_LR    OOF_Pred  \
0   4    784.983582    790.468576    764.757529   667.492085  591.994617   
1  15    863.431091    844.729374    862.122012   779.015390  573.931354   
2  21    715.363892    695.666440    778.202261   654.119606  573.931354   
3  30    792.706299    835.328327    803.697458   789.055631  602.764959   
4  34    764.853027    791.740481    777.046047   649.752725  591.994617   

   Actual  
0  2022.0  
1   849.0  
2  2670.0  
3   641.0  
4  2152.0  


[I 2024-12-29 17:02:31,671] Trial 1 finished with value: 1.0822750572960136 and parameters: {'w1': 0.04131409594219415, 'w2': 0.14532568798215173, 'w3': 0.10020261501430472, 'w4': 0.07361429077532888, 'w5': 0.9320517918770352}. Best is trial 0 with value: 1.0599131324428062.
[I 2024-12-29 17:02:31,824] Trial 2 finished with value: 1.0554334217872485 and parameters: {'w1': 0.19784386862153291, 'w2': 0.5825105235860202, 'w3': 0.9852792614479068, 'w4': 0.6381664518015687, 'w5': 0.50186966187385}. Best is trial 2 with value: 1.0554334217872485.
[I 2024-12-29 17:02:31,977] Trial 3 finished with value: 1.0599624702466421 and parameters: {'w1': 0.28276405675694605, 'w2': 0.280310826518679, 'w3': 0.1139425930399135, 'w4': 0.5289661329820987, 'w5': 0.1247008544257231}. Best is trial 2 with value: 1.0554334217872485.
[I 2024-12-29 17:02:32,132] Trial 4 finished with value: 1.0574305602279381 and parameters: {'w1': 0.007072064594969341, 'w2': 0.19533194820169064, 'w3': 0.1239392667926893, 'w4': 0

Best trial:
  Best RMSLE:  1.045661155273142
  Best Weights:
    XGB: 0.9616306197778753
    LGB: 0.38229532677222067
    Cat: 0.07946693182664182
    LR: 0.0002491984117771226
    GB: 0.0010527193699988169

Final DataFrame with Weighted Prediction:
              ID  OOF_Pred_XGB  OOF_Pred_LGB  OOF_Pred_cat  OOF_Pred_LR  \
0              4    784.983582    790.468576    764.757529   667.492085   
1             15    863.431091    844.729374    862.122012   779.015390   
2             21    715.363892    695.666440    778.202261   654.119606   
3             30    792.706299    835.328327    803.697458   789.055631   
4             34    764.853027    791.740481    777.046047   649.752725   
...          ...           ...           ...           ...          ...   
1199995  1199968    760.157654    781.762369    754.602342   715.841879   
1199996  1199969    685.693115    722.890318    757.323502   718.813552   
1199997  1199974    906.755920    920.120146    915.935565   845.142886   


In [43]:
weights

[0.6749730695800777,
 0.26833489376322944,
 0.0557782144224244,
 0.00017491354109599512,
 0.000738908693172267]

In [47]:
XGB =  weights[0]
LGB =  weights[1]
Cat = weights[2]
LR  = weights[3]
GB  = weights[4]


In [49]:
# Compute the weighted predictions
weighted_predicted = (
    (combined_df["xgb_pred"] * weights[0]) +
    (combined_df["lgb_pred"] * weights[1]) +
    (combined_df["cat_pred"] * weights[2]) +
    (combined_df["lr_pred"] * weights[3]) +
    (combined_df["gb_pred"] * weights[4])
)

In [50]:
combined_df['weighted_prediction'] = weighted_predicted
combined_df.head()

,id,xgb_pred,lgb_pred,cat_pred,lr_pred,gb_pred,final_pred,avg_predictions,weighted_prediction
0,1200000,714.507959,718.528380,778.918317,907.222200,415.584163,684.995516,706.952204,718.992304
1,1200001,829.446521,863.284798,784.389178,582.835247,604.458277,723.245153,732.882804,835.803912
2,1200002,785.417798,799.298493,785.537065,674.658230,588.745439,721.775192,726.731405,788.984429
3,1200003,804.212048,789.131510,782.219267,643.867759,588.745439,716.010727,721.635205,798.751439
4,1200004,773.894006,778.381459,764.650581,696.832252,571.045783,712.248657,716.960816,774.419199


In [51]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
sub['Premium Amount'] = combined_df['weighted_prediction']
sub.to_csv('submission(03).csv', index=False)
sub.head()

,id,Premium Amount
0,1200000,718.992304
1,1200001,835.803912
2,1200002,788.984429
3,1200003,798.751439
4,1200004,774.419199
